<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/TimeSeries/TimeSeries-2025/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D1%8B%D0%B9_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D1%83%D0%BC_%E2%84%96_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # 🧪 **Лабораторный практикум № 5**  
## **Сравнительный анализ Deep Learning моделей для прогнозирования временных рядов**

**Кафедра:** Кафедра анализа данных и технологий программирования  
**Дисциплина:** Машинное обучение в задачах прогнозирования  
**Уровень:** Магистратура, 2 курс  
**Преподаватель:** Арабов Муллошараф Курбонович  
**Консультации:** 21.09.2025  

---

## 🎯 1. Цели и задачи работы

### **Цель:**  
Освоить **глубокое обучение для прогнозирования временных рядов**, включая современные архитектуры — от базовых (MLP, RNN) до интерпретируемых (N-BEATS, TFT) и трансформеров. Научиться систематически оценивать **баланс точности, вычислительной сложности, стабильности и интерпретируемости**, а также строить обоснованные рекомендации по выбору архитектуры под конкретную задачу.

### **Задачи:**
1. Провести **предобработку данных**: нормализация, трансформации (лог, Бокс–Кокс), создание последовательностей.
2. Выполнить **инжиниринг признаков**: временные фичи, лаги, скользящие статистики, seasonal Fourier features, positional encoding.
3. Построить и сравнить следующие модели:
   - **Базовые**: MLP, TCN, N-BEATS, N-HiTS  
   - **Рекуррентные**: RNN, LSTM, GRU, BiLSTM, BiGRU  
   - **Трансформеры**: Transformer, Informer, Autoformer, PatchTST, TFT  
   - **Гибридные**: CNN-LSTM, CNN-GRU, TCN-Attention, LSTM-AE  
   - **Бейзлайны**: DLinear, NLinear, Naive, Seasonal Naive  
4. Настроить **гиперпараметры** по единым сеткам (вход, горизонт, LR, dropout и др.).
5. Применить **стратегии обучения**: early stopping, ReduceLROnPlateau, gradient clipping, разные оптимизаторы и функции потерь.
6. Провести **валидацию**: временные разбиения (70/15/15), expanding window CV, multi-horizon тестирование.
7. Реализовать **регуляризацию**: dropout, batch norm, weight decay, label smoothing.
8. Оценить модели по **масштабированным метрикам** и **статистическим тестам** (Diebold–Mariano, Wilcoxon).
9. Провести **диагностику**: learning curves, attention maps, N-BEATS компоненты, residual analysis.
10. Составить **сводную таблицу сравнения** по точности, скорости, стабильности, интерпретируемости.
11. Сформулировать **практические рекомендации** по выбору архитектуры для продакшена.

> 💡 **Важно:** работа фокусируется **исключительно на глубоком обучении** — классические статистические и ML-модели **не рассматриваются**.

> 📌 **Требования к данным:**  
> - Объём: **≥1000 наблюдений** (для устойчивого обучения DL)  
> - Горизонты прогноза: **h ∈ {24, 48, 168}** (1–7 дней)  
> - Поддержка **многомерных рядов** (для TFT, Informer и др.)

---

## 📚 2. Теоретические предпосылки

Глубокие модели позволяют улавливать **нелинейные зависимости**, **долгосрочные паттерны** и **взаимодействия между экзогенными переменными**, но требуют тщательной настройки и большого объёма данных.

- **MLP/TCN** — эффективны при коротких зависимостях; TCN обладает параллелизацией и receptive field.
- **RNN/LSTM/GRU** — классика для временных рядов, но страдают от затухающего градиента и медленного обучения.
- **N-BEATS/N-HiTS** — интерпретируемые, декомпозируют ряд на тренд и сезонность без предположений.
- **Трансформеры** — за счёт attention улавливают **long-range зависимости**, но чувствительны к шуму и требуют positional encoding.
- **TFT** — сочетает attention, статические/динамические фичи и **встроенную интерпретируемость** через variable selection.
- **Гибриды** — объединяют сильные стороны CNN (локальные паттерны) и RNN (временные зависимости).

> 🔑 **Ключевые принципы**:  
> - Никакого перемешивания данных во времени!  
> - Горизонт прогноза влияет на выбор архитектуры.  
> - Интерпретируемость ≠ точность, но критична в бизнесе.

---

## 🧪 3. Методика эксперимента и порядок выполнения

### **3.1. Этап 1. Предобработка временных рядов**

- **Трансформации**:  
  - Логарифм (если y > 0)  
  - **Бокса–Кокса**: `λ = boxcox_normmax(y_train)` → обратное преобразование при оценке  
- **Нормализация**:  
  - `MinMaxScaler` — для моделей с ограниченным диапазоном активаций  
  - `StandardScaler` — для моделей с гауссовыми предположениями  
- **Создание окон**:  
  ```python
  X, y = create_sequences(scaled_data, lookback=336, horizon=48)
  ```
- **Разбиение**: хронологическое — **train: 70%**, **val: 15%**, **test: 15%**

---

### **3.2. Этап 2. Инжиниринг признаков для DL**

- **Временные признаки**: `day_of_week`, `month`, `is_holiday` → one-hot или циклическое кодирование  
- **Fourier seasonal features**: `sin(2πt/s)`, `cos(2πt/s)` для s = 7, 12, 24  
- **Лаги и статистики**:  
  - Лаги: 1, 7, 30  
  - Скользящие: mean/std/min/max по окнам 7, 30  
- **Positional encoding** для трансформеров:  
  ```python
  pos_enc = PositionalEncoding(d_model, max_len=lookback)
  ```
- **Static covariates** (для TFT): например, тип товара, регион → вектор фиксированной длины

---

### **3.3. Этап 3. Конфигурация и обучение моделей**

| Группа | Архитектура | Ключевые параметры |
|--------|-------------|-------------------|
| **Базовые** | MLP, TCN, N-BEATS, N-HiTS | hidden layers, kernel size, stack types |
| **RNN** | LSTM, GRU, BiLSTM | hidden_size ∈ [64, 128, 256], num_layers ∈ [2, 4] |
| **Трансформеры** | Transformer, Informer, TFT | nhead ∈ [4,8], num_layers ∈ [2,3], dim_feedforward ∈ [256,512] |
| **Гибриды** | CNN-LSTM, TCN-Attention | CNN filters + RNN/Attention heads |
| **Бейзлайны** | DLinear, NLinear, Naive | Линейные проекции или просто последнее значение |

> Все модели обучены с:
> - **Batch size**: 32–128  
> - **LR**: 1e-4 — 1e-2  
> - **Dropout**: 0.1–0.3  
> - **Loss**: MSE + MAE + Huber  
> - **Оптимизаторы**: Adam, AdamW, RAdam  
> - **Early stopping**: patience=15  
> - **Gradient clipping**: norm ≤ 1.0

---

### **3.4. Этап 4. Валидация и регуляризация**

- **Expanding window CV**: [0:t] → [t+1:t+h] для 5 фолдов  
- **Multi-horizon**: оценка на h=24, 48, 168 отдельно  
- **Robustness**: тест на разных временных сегментах (зима/лето, до/после кризиса)  
- **Регуляризация**:  
  - Dropout, BatchNorm  
  - Weight decay (λ=1e-4)  
  - Label smoothing (ε=0.1)  
  - Gradient noise (σ=0.01)

---

### **3.5. Этап 5. Оценка качества**

- **Метрики**: MAE, RMSE, MAPE, **MASE** (главный), **RMSLE**, **SMAPE**, **R²**, **MDA**  
- **Статистические тесты**:  
  - **Diebold–Mariano**: значимость различий между моделями  
  - **Wilcoxon signed-rank**: непараметрическое сравнение пар  
- **Сводная таблица**:
  | Модель | MAE | RMSE | MAPE | MASE | Время | Память | Стабильность | Интерпретируемость |
  |--------|-----|------|------|------|--------|--------|--------------|-------------------|

---

### **3.6. Этап 6. Диагностика и интерпретация**

- **Learning curves**: train/val loss → признак переобучения?  
- **Attention maps** (TFT, Transformer): какие временные шаги важны?  
- **N-BEATS backcast**: как модель "объясняет" прошлое? Какие блоки доминируют?  
- **Residual analysis**: ACF остатков, Q-Q plot → есть ли структура?  
- **Permutation importance**: как изменяется MAE при перемешивании фичи?  
- **Case studies**: визуализация лучших и худших прогнозов

---

### **3.7. Этап 7. Продвинутые эксперименты**

- **Ансамблирование**:  
  - Взвешенное усреднение (по MASE)  
  - Stacking с линейной мета-моделью  
  - Анализ разнообразия моделей  
- **Адаптивные подходы**:  
  - **Transfer learning**: дообучение TFT на новом ряду  
  - **Online fine-tuning**: поступление новых данных → частичное обновление весов  
  - **Multi-task learning**: совместное обучение на связанных рядах (например, категории товаров)

---

### **3.8. Этап 8. Итоговый анализ и рекомендации**

1. **Ранжирование архитектур**:
   - **Точность**: N-HiTS ≈ TFT > PatchTST > LSTM  
   - **Интерпретируемость**: N-BEATS, TFT — лидеры  
   - **Скорость**: DLinear, MLP — самые быстрые  
   - **Стабильность**: TCN, N-HiTS — меньше вариации между запусками  

2. **Trade-offs**:  
   - Для **univariate** с трендом → **N-BEATS**  
   - Для **multivariate** с экзогенными → **TFT**  
   - Для **очень длинных горизонтов** → **Informer, PatchTST**  
   - Для **ограниченных ресурсов** → **DLinear, TCN**

3. **Гиперпараметры лучших моделей**:
   - TFT: hidden_size=128, num_heads=4, lookback=336  
   - N-HiTS: stack_types=["trend", "seasonality"], num_blocks=10  

4. **Визуализации**:  
   - Прогноз + факты на test  
   - Attention weights (TFT)  
   - N-BEATS trend/seasonality компоненты  

5. **Failure cases**:  
   - Резкие структурные сдвиги → все модели «теряются»  
   - Высокая волатильность → MAPE неинформативен  

6. **Продакшен-рекомендации**:  
   - Использовать **MASE** для мониторинга качества  
   - Настроить **автоматическое переобучение** раз в неделю  
   - Логировать **входные фичи и прогнозы** для аудита

---

## 🔍 4. Дополнительные задания (по желанию)

1. **Uncertainty quantification**: квантильная регрессия через TFT  
2. **Масштабирование на 1000+ рядов**: использование `pytorch-forecasting` или `darts`  
3. **Обучение без учителя**: LSTM-AE для аномалий → улучшение прогноза  
4. **Сравнение с AutoGluon-TS** как DL baseline  

---

## 📄 5. Требования к отчёту

1. Титульный лист  
2. Описание данных и источника  
3. Схема предобработки и feature engineering  
4. Гиперпараметры и конфигурации моделей  
5. Сводная таблица сравнения по трём горизонтам  
6. Графики: прогнозы, attention maps, N-BEATS компоненты, learning curves  
7. Результаты Diebold–Mariano и Wilcoxon тестов  
8. Анализ failure cases и ограничений  
9. Рекомендации по выбору архитектуры и продакшену  
10. Рефлексия

---

## 📊 6. Критерии оценки

| Оценка | Критерии |
|--------|---------|
| **5** | Все архитектуры реализованы. Есть полная диагностика, attention/N-BEATS визуализация, статистические тесты, ансамбли. Работа демонстрирует глубокое понимание trade-offs. |
| **4** | Реализованы 6–8 моделей. Есть сравнение, метрики, но без продвинутой интерпретации или одного из тестов. |
| **3** | Построены базовые DL-модели (MLP, LSTM, TFT). Есть MAE/RMSE, но без статистической проверки или диагностики. |
| **2** | Работа не выполнена или отсутствует сравнительный анализ.

---

## 📖 7. Литература

1. Hyndman, R.J., Athanasopoulos, G. *Forecasting: Principles and Practice*, 3rd ed. — [https://otexts.com/fpp3/](https://otexts.com/fpp3/)  
2. Oreshkin, B.N. et al. *N-BEATS: Neural basis expansion analysis for interpretable time series forecasting* — ICLR 2020  
3. Lim, B. et al. *Temporal Fusion Transformers for Interpretable Multi-horizon Time Series Forecasting* — IJF 2021  
4. Zhou, T. et al. *Informer: Beyond Efficient Transformer for Long Sequence Time-Series Forecasting* — AAAI 2021  
5. Woo, C.Y. et al. *N-HiTS: Neural Hierarchical Interpolation for Time Series Forecasting* — AAAI 2023  
6. PyTorch Forecasting Documentation — [https://pytorch-forecasting.readthedocs.io](https://pytorch-forecasting.readthedocs.io)  
7. Statsmodels, SciPy, Scikit-learn, Streamlit — официальная документация

> 💡 Эта работа завершает цикл по **методам прогнозирования** — от классических статистических до современных архитектур глубокого обучения. Полученные навыки лежат в основе **цифровых двойников**, **AI-преподавателей** и **адаптивных систем прогнозирования** в реальных прикладных задачах.
